In [1]:
!pip install spark-nlp
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.0/499.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=33f74503473684810e87e0357bd5c49ba5103d796f67ce58aaeae2412984c6c0
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession, DataFrame

In [3]:
import sparknlp

# Start Spark Session
spark = sparknlp.start()

In [4]:
spark = SparkSession.builder.appName("sentimentanalysis").getOrCreate()

In [5]:
!unzip "/content/drive/MyDrive/Big Data Model/model.zip"

Archive:  /content/drive/MyDrive/Big Data Model/model.zip
   creating: content/Model/
   creating: content/Model/stages/
   creating: content/Model/stages/1_REGEX_TOKENIZER_db011187f57d/
   creating: content/Model/stages/1_REGEX_TOKENIZER_db011187f57d/fields/
   creating: content/Model/stages/1_REGEX_TOKENIZER_db011187f57d/fields/rules/
 extracting: content/Model/stages/1_REGEX_TOKENIZER_db011187f57d/fields/rules/.part-00000.crc  
 extracting: content/Model/stages/1_REGEX_TOKENIZER_db011187f57d/fields/rules/._SUCCESS.crc  
  inflating: content/Model/stages/1_REGEX_TOKENIZER_db011187f57d/fields/rules/part-00001  
  inflating: content/Model/stages/1_REGEX_TOKENIZER_db011187f57d/fields/rules/part-00000  
 extracting: content/Model/stages/1_REGEX_TOKENIZER_db011187f57d/fields/rules/_SUCCESS  
 extracting: content/Model/stages/1_REGEX_TOKENIZER_db011187f57d/fields/rules/.part-00001.crc  
   creating: content/Model/stages/1_REGEX_TOKENIZER_db011187f57d/metadata/
 extracting: content/Model/st

In [6]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, regexp_replace, trim
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel
from sklearn.metrics import classification_report
import re
import json
import requests
import pandas as pd

url = "https://shopee.sg/3-Month-Warranty-116-Plus-Smart-Watch-Blood-Pressure-Heart-Rate-Monitor-Waterproof-Fitness-Tracker-Watch-Smart-Band-i.139268812.6250395408?sp_atk=450a151"

r = re.search(r"i\.(\d+)\.(\d+)", url)
shop_id, item_id = r[1], r[2]
ratings_url = "https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0"

offset = 0
d = {"username": [], "comment": []}
flag_stop = False  # Biến cờ để kiểm tra điều kiện dừng vòng lặp

while not flag_stop:
    data = requests.get(
        ratings_url.format(shop_id=shop_id, item_id=item_id, offset=offset)
    ).json()

    if "data" not in data or "ratings" not in data["data"]:
        print("Không có dữ liệu hoặc đã xảy ra lỗi.")
        break

    ratings = data["data"]["ratings"]
    if not ratings:
        print("Dữ liệu 'ratings' rỗng hoặc bằng None. Dừng lại và không lưu vào file CSV.")
        break

    for rating in ratings:
        comment_text = rating["comment"]

        if comment_text is not None:  # Kiểm tra xem trường "comment" có giá trị hợp lệ hay không (không là None)
            comment_text = comment_text.strip()

        d["username"].append(rating["author_username"])
        d["comment"].append(comment_text)

        # print(rating["author_username"])
        # print(comment_text)
        # print("-" * 100)

    offset += 20

# Lưu vào file CSV chỉ khi có dữ liệu có trường "comment" không rỗng
if d["comment"]:
    df = pd.DataFrame(d)
    df.to_csv("dataEN.csv", index=False)
else:
    print("Không có dữ liệu có trường 'comment' không rỗng để lưu vào file CSV.")

# Đọc dữ liệu từ file CSV vào DataFrame với tùy chọn multiLine=True để xử lý dữ liệu có nhiều dòng
df = spark.read.csv("dataEN.csv", header=True, multiLine=True, inferSchema=True)

# Đổi tên cột "comment" thành "short_review"
df = df.withColumnRenamed("comment", "short_review")

# Loại bỏ các dòng có trường "short_review" hoặc "username" rỗng từ DataFrame
df = df.dropna(subset=["short_review", "username"])
df = df.dropna(subset=["short_review"])
# Xoá các icon (ký tự không phải tiếng Anh) trong trường "short_review"
df = df.withColumn("short_review", regexp_replace("short_review", r"[^a-zA-Z\s]", " "))

# Thay thế các chuỗi gồm nhiều khoảng trắng liên tiếp bằng một khoảng trắng duy nhất
df = df.withColumn("short_review", regexp_replace("short_review", r"\s+", " "))

# Loại bỏ khoảng trắng ở đầu và cuối mỗi giá trị trong trường "short_review"
df = df.withColumn("short_review", trim(col("short_review")))

# Lọc các dòng có trường "short_review" không rỗng
df = df.filter(col("short_review") != "")


# Load model and motional analysis  --------------------------------------------------------------------
model_path = "/content/content/Model"
loaded_model = PipelineModel.load(model_path)
# Use the loaded model to make predictions on the new data
predictions = loaded_model.transform(df)
df = predictions.select(['short_review','label.result']).toPandas()
df['result'] = df['result'].apply(lambda x: x[0])

# Process name label
# Function to map 'pos' to 'positive' and 'neg' to 'negative'
def map_sentiment(sentiment):
    if sentiment == 'pos':
        return 'positive'
    elif sentiment == 'neg':
        return 'negative'
    else:
        return sentiment

# Applying the mapping function to the 'result' column
df['result'] = df['result'].apply(map_sentiment)

# Displaying the DataFrame
print(df)

positive_count = df[df['result'] == 'positive'].shape[0]
negative_count = df[df['result'] == 'negative'].shape[0]
print(f"Number of positive reviews: {positive_count}")
print(f"Number of negative reviews: {negative_count}")


# Saving the Pandas DataFrame to a CSV file
df.to_csv('predictions.csv', index=False)

Dữ liệu 'ratings' rỗng hoặc bằng None. Dừng lại và không lưu vào file CSV.
                                          short_review    result
0    Value For Money yes Performance press hold to ...  positive
1    Performance good Value For Money yes Best Feat...  positive
2    Best Feature s very good Value For Money excel...  positive
3    Super fast delivery Received item one week aft...  positive
4    I like it fast delivery in good condition easy...  positive
..                                                 ...       ...
597                                            Love it  positive
598                                Thanks u the seller  positive
599                                               good  positive
600        Item received and it works thank you seller  positive
601                                              ok ok  positive

[602 rows x 2 columns]
Number of positive reviews: 377
Number of negative reviews: 225
